In [1]:
%pylab inline
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)
from IPython.display import clear_output
import time
#from time import sleep
import numpy as np
import pandas as pd
from sklearn import linear_model
import math
import gym
import myfrozen
import pickle
import img_utils
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import print_function, division

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, Lambda
from keras.layers.advanced_activations import LeakyReLU, Softmax
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras import backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [3]:
def get_data(dictionary):
    X_train = np.zeros((100,64))
    i = 0
    for k, v in dictionary.items():
        for kp, vp in v.items():
            if kp == 'policy':
                X_train[i,:] = vp
                i += 1
    return X_train

def load_all_data(foldername):
    all_data = np.empty((0,64))
    for filename in os.listdir(foldername):
        if filename.endswith((".pkl")):
            x = os.path.join(foldername,filename)
            full = pickle.load(open(x,'rb'))
            X_temp = get_data(full)
            all_data = np.concatenate((all_data, X_temp))
    np.random.seed(123)
    np.random.shuffle(all_data)
    return all_data

In [4]:
X_train = load_all_data('saved')

/anaconda/envs/drl/lib/python2.7/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearRegression from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
X_train.shape

(800, 64)

In [6]:
import h5py

## GAN

In [9]:
class GAN():
    def __init__(self):
        self.img_rows = 64
        self.img_cols = 4
        self.img_shape = (self.img_rows, self.img_cols)
        self.latent_dim = 64

        optimizer = Adam(0.0002)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        #self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
    def build_generator(self):

        model = Sequential()
        
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='relu'))
        model.add(Reshape(self.img_shape))
        model.add(Lambda(lambda x: K.tf.nn.softmax(x)))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    
    def train(self, epochs, batch_size=128, sample_interval=50):
    
        D = {}
        #X_train = mix_data()
        X_train = load_all_data('saved')
        X_train = to_categorical(X_train)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)
            # Train the discriminator 
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                D = self.sample_images(epoch,D)
                
        self.generator.save('models/policies/generator.h5')
        self.discriminator.save('models/policies/discriminator.h5')
        return D
                
    def sample_images(self, epoch, D):
                    
        def show_policy(q):
            A2A=['<','v','>','^']
            grid = np.zeros(64, dtype='<U2')
            for index in xrange(64):
                action = np.argmax(q[index,:])
                grid[index] = A2A[action]
            grid = np.reshape(grid, (8, 8), order='C')
            print(grid)
            return grid
        
        #r, c = 1, 1
        for iteration in range(5):
            noise = np.random.normal(0, 1, (1, self.latent_dim))
            gen_imgs = self.generator.predict(noise)        
            D[str(epoch)] = {}
            D[str(epoch)][str(iteration)] = gen_imgs
            _ = show_policy(np.squeeze(gen_imgs, axis=0))
        
        return D

In [10]:
gan = GAN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               65792     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
Total params: 66,049
Trainable params: 66,049
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 256)               16640     
_________________________________________________________________
leaky_

In [11]:
# Without training discriminator (printing POLICY)
Dtest = gan.train(epochs=10000, batch_size=30, sample_interval=500)

/anaconda/envs/drl/lib/python2.7/site-packages/keras/engine/training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.658176, acc.: 38.33%] [G loss: 0.603992]
[[u'^' u'v' u'^' u'^' u'<' u'<' u'v' u'^']
 [u'<' u'^' u'^' u'<' u'<' u'v' u'^' u'<']
 [u'v' u'^' u'v' u'^' u'v' u'>' u'<' u'<']
 [u'^' u'>' u'^' u'>' u'^' u'<' u'^' u'v']
 [u'<' u'<' u'^' u'<' u'<' u'>' u'v' u'^']
 [u'^' u'v' u'v' u'<' u'<' u'>' u'^' u'v']
 [u'^' u'^' u'<' u'^' u'v' u'>' u'<' u'>']
 [u'^' u'>' u'v' u'^' u'<' u'v' u'^' u'^']]
[[u'<' u'<' u'<' u'>' u'<' u'^' u'<' u'^']
 [u'<' u'^' u'>' u'<' u'>' u'>' u'^' u'^']
 [u'v' u'>' u'^' u'^' u'v' u'>' u'<' u'<']
 [u'>' u'>' u'v' u'v' u'<' u'<' u'v' u'v']
 [u'<' u'<' u'^' u'<' u'>' u'>' u'<' u'v']
 [u'^' u'<' u'>' u'v' u'<' u'v' u'^' u'^']
 [u'^' u'^' u'<' u'>' u'v' u'>' u'>' u'>']
 [u'v' u'^' u'>' u'^' u'<' u'>' u'<' u'<']]
[[u'^' u'v' u'>' u'<' u'v' u'>' u'v' u'v']
 [u'>' u'>' u'v' u'v' u'>' u'>' u'^' u'>']
 [u'<' u'<' u'v' u'<' u'v' u'<' u'^' u'<']
 [u'<' u'v' u'>' u'<' u'>' u'v' u'<' u'v']
 [u'^' u'v' u'<' u'^' u'<' u'>' u'>' u'v']
 [u'v' u'>' u'v' u'^' u'v' u'^' u'<' u'v'

119 [D loss: 0.453784, acc.: 88.33%] [G loss: 1.014173]
120 [D loss: 0.473347, acc.: 86.67%] [G loss: 1.104575]
121 [D loss: 0.465975, acc.: 81.67%] [G loss: 1.012266]
122 [D loss: 0.534255, acc.: 76.67%] [G loss: 1.064459]
123 [D loss: 0.434972, acc.: 91.67%] [G loss: 1.052075]
124 [D loss: 0.470021, acc.: 86.67%] [G loss: 1.061056]
125 [D loss: 0.404524, acc.: 90.00%] [G loss: 1.064534]
126 [D loss: 0.456278, acc.: 86.67%] [G loss: 1.039688]
127 [D loss: 0.459018, acc.: 86.67%] [G loss: 1.024419]
128 [D loss: 0.475444, acc.: 86.67%] [G loss: 1.057029]
129 [D loss: 0.470103, acc.: 88.33%] [G loss: 1.052738]
130 [D loss: 0.425434, acc.: 90.00%] [G loss: 1.054986]
131 [D loss: 0.455187, acc.: 91.67%] [G loss: 1.025653]
132 [D loss: 0.447218, acc.: 86.67%] [G loss: 1.060123]
133 [D loss: 0.426112, acc.: 90.00%] [G loss: 1.153520]
134 [D loss: 0.532417, acc.: 80.00%] [G loss: 1.091717]
135 [D loss: 0.430569, acc.: 93.33%] [G loss: 1.089996]
136 [D loss: 0.454600, acc.: 90.00%] [G loss: 1.

266 [D loss: 0.503083, acc.: 80.00%] [G loss: 1.109034]
267 [D loss: 0.515383, acc.: 81.67%] [G loss: 1.140026]
268 [D loss: 0.504154, acc.: 75.00%] [G loss: 1.142755]
269 [D loss: 0.451781, acc.: 81.67%] [G loss: 1.025459]
270 [D loss: 0.541141, acc.: 76.67%] [G loss: 1.206217]
271 [D loss: 0.496314, acc.: 86.67%] [G loss: 1.250384]
272 [D loss: 0.463175, acc.: 86.67%] [G loss: 1.158693]
273 [D loss: 0.492052, acc.: 85.00%] [G loss: 1.161798]
274 [D loss: 0.584357, acc.: 68.33%] [G loss: 1.165421]
275 [D loss: 0.457185, acc.: 85.00%] [G loss: 1.198434]
276 [D loss: 0.493480, acc.: 80.00%] [G loss: 1.111568]
277 [D loss: 0.446362, acc.: 86.67%] [G loss: 1.029360]
278 [D loss: 0.483061, acc.: 80.00%] [G loss: 1.003360]
279 [D loss: 0.457990, acc.: 85.00%] [G loss: 1.024891]
280 [D loss: 0.485524, acc.: 83.33%] [G loss: 0.978594]
281 [D loss: 0.488118, acc.: 76.67%] [G loss: 0.995805]
282 [D loss: 0.497814, acc.: 83.33%] [G loss: 0.942417]
283 [D loss: 0.575024, acc.: 71.67%] [G loss: 1.

415 [D loss: 0.531767, acc.: 75.00%] [G loss: 1.057742]
416 [D loss: 0.538503, acc.: 76.67%] [G loss: 1.014107]
417 [D loss: 0.535922, acc.: 76.67%] [G loss: 1.082986]
418 [D loss: 0.451029, acc.: 85.00%] [G loss: 1.108738]
419 [D loss: 0.456734, acc.: 86.67%] [G loss: 1.128935]
420 [D loss: 0.438698, acc.: 90.00%] [G loss: 1.083821]
421 [D loss: 0.503644, acc.: 81.67%] [G loss: 1.139723]
422 [D loss: 0.456098, acc.: 83.33%] [G loss: 1.185123]
423 [D loss: 0.435575, acc.: 80.00%] [G loss: 1.168594]
424 [D loss: 0.480245, acc.: 75.00%] [G loss: 1.152092]
425 [D loss: 0.488568, acc.: 80.00%] [G loss: 1.211272]
426 [D loss: 0.402105, acc.: 85.00%] [G loss: 1.209663]
427 [D loss: 0.449888, acc.: 85.00%] [G loss: 1.156750]
428 [D loss: 0.529352, acc.: 68.33%] [G loss: 1.252660]
429 [D loss: 0.440784, acc.: 85.00%] [G loss: 1.148854]
430 [D loss: 0.510022, acc.: 83.33%] [G loss: 1.096049]
431 [D loss: 0.442890, acc.: 88.33%] [G loss: 1.079434]
432 [D loss: 0.400236, acc.: 90.00%] [G loss: 1.

...

9440 [D loss: 0.256020, acc.: 93.33%] [G loss: 1.612316]
9441 [D loss: 0.258667, acc.: 91.67%] [G loss: 1.831578]
9442 [D loss: 0.304841, acc.: 91.67%] [G loss: 1.668680]
9443 [D loss: 0.246449, acc.: 93.33%] [G loss: 1.662622]
9444 [D loss: 0.220058, acc.: 98.33%] [G loss: 1.599111]
9445 [D loss: 0.301383, acc.: 88.33%] [G loss: 1.524880]
9446 [D loss: 0.272264, acc.: 91.67%] [G loss: 1.643140]
9447 [D loss: 0.284264, acc.: 90.00%] [G loss: 1.543237]
9448 [D loss: 0.246838, acc.: 95.00%] [G loss: 1.535247]
9449 [D loss: 0.303508, acc.: 90.00%] [G loss: 1.784536]
9450 [D loss: 0.343398, acc.: 88.33%] [G loss: 1.562468]
9451 [D loss: 0.293041, acc.: 93.33%] [G loss: 1.569154]
9452 [D loss: 0.323543, acc.: 88.33%] [G loss: 1.536453]
9453 [D loss: 0.302126, acc.: 91.67%] [G loss: 1.487986]
9454 [D loss: 0.286115, acc.: 93.33%] [G loss: 1.651358]
9455 [D loss: 0.208928, acc.: 98.33%] [G loss: 1.584192]
9456 [D loss: 0.269625, acc.: 93.33%] [G loss: 1.742847]
9457 [D loss: 0.253719, acc.: 9

9558 [D loss: 0.343000, acc.: 86.67%] [G loss: 1.353822]
9559 [D loss: 0.331568, acc.: 88.33%] [G loss: 1.676784]
9560 [D loss: 0.326660, acc.: 85.00%] [G loss: 1.785569]
9561 [D loss: 0.356133, acc.: 90.00%] [G loss: 1.559992]
9562 [D loss: 0.373648, acc.: 83.33%] [G loss: 1.564471]
9563 [D loss: 0.351925, acc.: 90.00%] [G loss: 1.574404]
9564 [D loss: 0.352031, acc.: 90.00%] [G loss: 1.554454]
9565 [D loss: 0.291197, acc.: 90.00%] [G loss: 1.662750]
9566 [D loss: 0.325361, acc.: 91.67%] [G loss: 1.647083]
9567 [D loss: 0.327725, acc.: 91.67%] [G loss: 1.510539]
9568 [D loss: 0.310520, acc.: 91.67%] [G loss: 1.556873]
9569 [D loss: 0.346629, acc.: 90.00%] [G loss: 1.740556]
9570 [D loss: 0.285466, acc.: 91.67%] [G loss: 1.746578]
9571 [D loss: 0.323885, acc.: 90.00%] [G loss: 1.796131]
9572 [D loss: 0.347333, acc.: 90.00%] [G loss: 1.648067]
9573 [D loss: 0.304482, acc.: 93.33%] [G loss: 1.574754]
9574 [D loss: 0.345428, acc.: 88.33%] [G loss: 1.691896]
9575 [D loss: 0.324459, acc.: 9

9705 [D loss: 0.249443, acc.: 95.00%] [G loss: 1.701738]
9706 [D loss: 0.221961, acc.: 95.00%] [G loss: 1.713441]
9707 [D loss: 0.245425, acc.: 95.00%] [G loss: 1.637226]
9708 [D loss: 0.346956, acc.: 90.00%] [G loss: 1.650914]
9709 [D loss: 0.315997, acc.: 91.67%] [G loss: 1.668692]
9710 [D loss: 0.308863, acc.: 91.67%] [G loss: 1.611863]
9711 [D loss: 0.249178, acc.: 93.33%] [G loss: 1.623490]
9712 [D loss: 0.324589, acc.: 86.67%] [G loss: 1.574413]
9713 [D loss: 0.293050, acc.: 93.33%] [G loss: 1.621447]
9714 [D loss: 0.275494, acc.: 95.00%] [G loss: 1.802209]
9715 [D loss: 0.300977, acc.: 90.00%] [G loss: 1.719487]
9716 [D loss: 0.306896, acc.: 98.33%] [G loss: 1.640094]
9717 [D loss: 0.270555, acc.: 95.00%] [G loss: 1.637850]
9718 [D loss: 0.292797, acc.: 93.33%] [G loss: 1.709806]
9719 [D loss: 0.278502, acc.: 95.00%] [G loss: 1.793676]
9720 [D loss: 0.356841, acc.: 88.33%] [G loss: 1.683836]
9721 [D loss: 0.370704, acc.: 88.33%] [G loss: 1.578864]
9722 [D loss: 0.361908, acc.: 8

9851 [D loss: 0.316613, acc.: 88.33%] [G loss: 1.634194]
9852 [D loss: 0.328864, acc.: 86.67%] [G loss: 1.710482]
9853 [D loss: 0.346123, acc.: 86.67%] [G loss: 1.553237]
9854 [D loss: 0.312644, acc.: 93.33%] [G loss: 1.646082]
9855 [D loss: 0.301001, acc.: 86.67%] [G loss: 1.735988]
9856 [D loss: 0.329319, acc.: 91.67%] [G loss: 1.523050]
9857 [D loss: 0.320535, acc.: 91.67%] [G loss: 1.556077]
9858 [D loss: 0.255234, acc.: 98.33%] [G loss: 1.515678]
9859 [D loss: 0.257743, acc.: 93.33%] [G loss: 1.380411]
9860 [D loss: 0.283253, acc.: 95.00%] [G loss: 1.432395]
9861 [D loss: 0.256605, acc.: 98.33%] [G loss: 1.610035]
9862 [D loss: 0.275810, acc.: 96.67%] [G loss: 1.436195]
9863 [D loss: 0.256048, acc.: 93.33%] [G loss: 1.536003]
9864 [D loss: 0.239380, acc.: 95.00%] [G loss: 1.595752]
9865 [D loss: 0.307348, acc.: 96.67%] [G loss: 1.675195]
9866 [D loss: 0.320864, acc.: 93.33%] [G loss: 1.870100]
9867 [D loss: 0.291790, acc.: 91.67%] [G loss: 1.797948]
9868 [D loss: 0.287871, acc.: 9

9998 [D loss: 0.310762, acc.: 93.33%] [G loss: 1.520724]
9999 [D loss: 0.313954, acc.: 91.67%] [G loss: 1.582917]


In [12]:
from keras.models import load_model

In [13]:
generator = load_model('models/policies/generator.h5')

/anaconda/envs/drl/lib/python2.7/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [14]:
def show_policy(q):
    A2A=['<','v','>','^']
    grid = np.zeros(64, dtype='<U2')
    for index in xrange(64):
        action = np.argmax(q[index,:])
        grid[index] = A2A[action]
    grid = np.reshape(grid, (8, 8), order='C')
    print(grid)
    return grid

for iteration in range(10):
    noise = np.random.normal(0, 1, (1, 64))
    gen_imgs = generator.predict(noise)
    _ = show_policy(np.squeeze(gen_imgs, axis=0))

[[u'v' u'v' u'v' u'v' u'v' u'<' u'<' u'<']
 [u'v' u'v' u'v' u'v' u'v' u'v' u'<' u'<']
 [u'v' u'v' u'v' u'v' u'v' u'v' u'v' u'<']
 [u'v' u'v' u'v' u'v' u'v' u'<' u'v' u'<']
 [u'v' u'<' u'v' u'v' u'<' u'<' u'<' u'<']
 [u'v' u'v' u'v' u'v' u'<' u'<' u'<' u'<']
 [u'v' u'v' u'v' u'v' u'<' u'v' u'<' u'v']
 [u'>' u'>' u'>' u'>' u'<' u'<' u'<' u'<']]
[[u'<' u'<' u'<' u'<' u'<' u'<' u'<' u'<']
 [u'^' u'<' u'<' u'<' u'<' u'<' u'<' u'<']
 [u'^' u'<' u'<' u'<' u'<' u'<' u'<' u'<']
 [u'^' u'<' u'<' u'^' u'<' u'<' u'<' u'<']
 [u'^' u'<' u'<' u'^' u'<' u'<' u'<' u'<']
 [u'^' u'<' u'<' u'^' u'<' u'<' u'<' u'<']
 [u'^' u'<' u'<' u'<' u'<' u'<' u'<' u'<']
 [u'^' u'<' u'<' u'<' u'<' u'<' u'<' u'<']]
[[u'v' u'<' u'v' u'v' u'v' u'<' u'<' u'<']
 [u'v' u'v' u'<' u'v' u'v' u'v' u'<' u'<']
 [u'v' u'v' u'<' u'v' u'<' u'v' u'v' u'<']
 [u'v' u'v' u'<' u'v' u'<' u'<' u'<' u'<']
 [u'v' u'<' u'v' u'v' u'<' u'<' u'<' u'<']
 [u'v' u'<' u'v' u'v' u'<' u'<' u'<' u'v']
 [u'v' u'<' u'v' u'v' u'<' u'<' u'<' u'v']
 [u'>' u'

# TESTING IDEAS